In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
##########################Load Libraries  ####################################
import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import preprocessing, metrics
from ipywidgets import widgets, interactive
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta 
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
from itertools import cycle
import datetime as dt
from torch.autograd import Variable
import random 
import os
from matplotlib.pyplot import figure
from fastprogress import master_bar, progress_bar
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time 
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import torch 
from numpy import array
%matplotlib inline

#from gensim.models import Word2Vec
#import gensim.downloader as api

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

# Univariate LSTM Models
LSTMs can be used to model univariate time series forecasting problems.

These are problems comprised of a single series of observations and a model is required to learn from the series of past observations to predict the next value in the sequence.

We will demonstrate a number of variations of the LSTM model for univariate time series forecasting.

This section is divided into six parts; they are:

    1. Data Preparation
    2. Vanilla LSTM
    3. Stacked LSTM
    4. Bidirectional LSTM
    5. CNN LSTM
    6. ConvLSTM
Each of these models are demonstrated for one-step univariate time series forecasting, but can easily be adapted and used as the input part of a model for other types of time series forecasting problems.

## 1. Data Preparation

In [3]:
# Consider a given univariate sequence:
[10, 20, 30, 40, 50, 60, 70, 80, 90]

[10, 20, 30, 40, 50, 60, 70, 80, 90]

We can divide the sequence into multiple input/output patterns called samples, where three time steps are used as input and one time step is used as output for the one-step prediction that is being learned.

In [4]:
# The split_sequence() function below implements this behavior 
# and will split a given univariate sequence into multiple samples where each sample has a specified number of time steps and the output is a single time step.
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [5]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [6]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# define model
n_features = 1
model = Sequential()
model.add(LSTM(50,activation='relu',input_shape=(n_steps,n_features)))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [8]:
# reshape from[sample,timesteps] into [samples,timesteps,features]
n_features = 1
X = X.reshape((X.shape[0],X.shape[1],n_features))

In [9]:
# fit model
model.fit(X,y,epochs =200, verbose=0)

In [10]:
# demostration prediction
x_input = array([70,80,90])
x_input = x_input.reshape(1,n_steps,n_features)
yhat = model.predict(x_input,verbose=0)

In [11]:
print(yhat)

[[103.00992]]


## 2.Stacked LSTM

In [12]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [13]:
# fit model
model.fit(X,y,epochs =200, verbose=0)

In [14]:
# demostration prediction
x_input = array([70,80,90])
x_input = x_input.reshape(1,n_steps,n_features)
yhat = model.predict(x_input,verbose=0)

In [15]:
print(yhat)

[[103.606125]]


## 3.Bidirectional LSTM

In [16]:
from keras.layers import Bidirectional

In [17]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [18]:
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


In [19]:
print(yhat)

[[101.80447]]


## 4.CNN LSTM

In [20]:
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [21]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


In [22]:
print(yhat)

[[100.56886]]


## 5.ConvLSTM

In [23]:
from keras.layers import Flatten
from keras.layers import ConvLSTM2D

In [24]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [25]:
print(yhat)

[[103.93673]]


In [48]:
#neural network
from numpy import exp, array, random, dot

In [49]:
class NeuralNetwork():
    def __init__(self):
        # Seed the random nimber generator, so it generates the same numbers
        # every time the program runs.
        random.seed(1)
        
        # We model a single neuron, with 3 output connections and 1 output connection
        # We assign random weights to a 3x1 matrix, with values in the range -1 to 1
        # and mean 0.
        self.synaptic_weights = 2*random.random((3,1))-1
        
    # define sigmoid - activate function
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_derivative(self,x):
        return np.exp(-x)/(1+np.exp(-x))**2
        
    # We train the neural network through a process o trial and error.
    # Adjusting the synaptic weights each time.
    def train(self,training_set_input,training_set_output,num_of_traning_iterations):
        for i in range(num_of_traning_iterations):
            output = self.activate_func(training_set_input)
            
            # calculate the error (The different between the desied output
            # and the predicted output)
            
            error = training_set_output-output
            
            # Multiply the error by the input and again by the gradient of the Sigmoid curve
            # This means less confident weight are adjusted more.
            # This means input, which are zero, do not cause changes to the weight.
            adjustment = dot(training_set_input.T,error*self.sigmoid_derivative(output))
            
            # Adjust the weight
            self.synaptic_weights += adjustment
    def activate_func(self,inputs):
        return self.sigmoid(dot(inputs,self.synaptic_weights))


In [51]:
 # Intialise a signle neural network.
neural_network = NeuralNetwork()
    
print("Random starting synaptic weights:")
print(neural_network.synaptic_weights)
    
# The training set: 4 examples, each consisting of 3 input values
# and 1 output values.
    
training_set_input = array([[0,0,1],
                               [1,1,1],
                               [1,0,1],
                               [0,1,1]])
training_set_output = array([[0,1,1,0]]).T
    
# Train the neural network  using a training set.
# Do it 10,000 times and make samll adjustments each times.
neural_network.train(training_set_input,training_set_output,100)
    
print("Mew synaptic weights after training:")
print(neural_network.synaptic_weights)
    
# Test the neural network with a new situation.
print("considering new situation[1,0,0]->?:")
print(neural_network.activate_func(array([1,0,0])))

Random starting synaptic weights:
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
Mew synaptic weights after training:
[[19.64772187]
 [-0.40556114]
 [-9.53843554]]
considering new situation[1,0,0]->?:
[1.]


In [34]:
a = [[1,2,3],
    [4,5,6],
    [7,8,9]]
b = [[1,1,1],
    [1,1,1],
    [1,1,1]]

In [35]:
np.dot(a,b)

array([[ 6,  6,  6],
       [15, 15, 15],
       [24, 24, 24]])